In [1]:
import cv2 as cv
import dlib
import numpy as np
from PIL import Image
from io import BytesIO
import requests
import face_recognition

In [ ]:
class FaceVerificationSystem:
    def __init__(self,
                 reference_image_url,
                 shape_predictor_path='shape_predictor_68_face_landmarks.dat',
                 face_match_threshold=0.6, #lower means stricter
                 eye_ar_threshold=0.25, #lower means eyes are closed
                 eye_ar_consec_frames=3, #no of consecutive frame below the threshold to count as a blink
                 required_blinks=1):

        self.reference_image_url = reference_image_url
        self.face_match_threshold = face_match_threshold
        self.shape_predictor_path = shape_predictor_path
        self.eye_ar_threshold = eye_ar_threshold
        self.eye_ar_consec_frames = eye_ar_consec_frames
        self.required_blinks = required_blinks

        self.counter = 0
        self.total_blinks = 0
        self.liveness_passed = False
        self.verification_passed = False

        self.known_face_encodings = []
        self.known_face_names = []

        self.predictor = dlib.shape_predictor(self.shape_predictor_path)
        self.detector = dlib.get_frontal_face_detector()

    def eye_aspect_ratio(self, eye):
        A = np.linalg.norm(eye[1] - eye[5])
        B = np.linalg.norm(eye[2] - eye[4])
        C = np.linalg.norm(eye[0] - eye[3])
        return (A + B) / (2.0 * C)

    def fetch_reference_image(self):
        try:
            print(f"Fetching reference image from: {self.reference_image_url}")
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(self.reference_image_url, headers=headers)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            return cv.cvtColor(np.array(img), cv.COLOR_RGB2BGR)
        except Exception as e:
            print(f"Error fetching reference image: {e}")
            return None

    def initialize_reference_encoding(self):
        reference_image_np = self.fetch_reference_image()
        if reference_image_np is None:
            raise ValueError("Reference image could not be loaded.")

        reference_image_rgb = cv.cvtColor(reference_image_np, cv.COLOR_BGR2RGB)
        ref_face_locations = face_recognition.face_locations(reference_image_rgb)
        if not ref_face_locations:
            raise ValueError("No face found in the reference image.")

        ref_face_encoding = face_recognition.face_encodings(reference_image_rgb, ref_face_locations)[0]
        self.known_face_encodings = [ref_face_encoding]
        self.known_face_names = ["Registered User"]

    def run_verification(self):
        self.initialize_reference_encoding()

        video_capture = cv.VideoCapture(0)
        if not video_capture.isOpened():
            raise RuntimeError("Error: Could not open webcam.")

        while True:
            ret, frame = video_capture.read()
            if not ret:
                print("Failed to grab frame.")
                break

            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            faces = self.detector(gray)

            display_text = "Status: Initializing..."
            text_color = (0, 255, 255)

            if len(faces) == 0:
                display_text = "Status: No face detected."
                text_color = (0, 0, 255)
                self.verification_passed = False
                self.liveness_passed = False
                self.total_blinks = 0
                self.counter = 0

            elif len(faces) > 1:
                display_text = "Status: Multiple persons detected!!"
                text_color = (0, 0, 255)
                self.verification_passed = False
                self.liveness_passed = False
                self.total_blinks = 0
                self.counter = 0

            else:
                shape = self.predictor(gray, faces[0])
                landmarks = np.array([(p.x, p.y) for p in shape.parts()])

                face_locations = face_recognition.face_locations(rgb)
                face_encodings = face_recognition.face_encodings(rgb, face_locations)

                if face_encodings:
                    face_encoding = face_encodings[0]
                    face_location = face_locations[0]

                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=self.face_match_threshold)
                    name = "Unknown User"

                    if True in matches:
                        name = self.known_face_names[matches.index(True)]
                        self.verification_passed = True
                        display_text = f"Verification: {name} (Match)"
                        text_color = (0, 255, 0)
                    else:
                        self.verification_passed = False
                        display_text = "User not Verified."
                        text_color = (0, 0, 255)
                        self.liveness_passed = False
                        self.total_blinks = 0
                        self.counter = 0

                    if self.verification_passed and not self.liveness_passed:
                        left_eye = landmarks[36:42]
                        right_eye = landmarks[42:48]
                        ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0

                        if ear < self.eye_ar_threshold:
                            self.counter += 1
                        else:
                            if self.counter >= self.eye_ar_consec_frames:
                                self.total_blinks += 1
                            self.counter = 0

                        if self.total_blinks >= self.required_blinks:
                            self.liveness_passed = True
                            text_color = (0, 255, 0)

                        cv.putText(frame, f"Liveness: Blinks {self.total_blinks}/{self.required_blinks}", (10, 80),
                                   cv.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

                    elif self.liveness_passed:
                        cv.putText(frame, "LIVENESS CHECK: PASSED!!", (10, 80),
                                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    # Draw bounding box
                    top, right, bottom, left = face_location
                    cv.rectangle(frame, (left, top), (right, bottom), text_color, 2)
                    cv.putText(frame, name, (left + 6, bottom - 6),
                               cv.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)

                    if self.verification_passed and self.liveness_passed:
                        display_text = "Verification and Liveness: PASSED!"
                        text_color = (0, 255, 0)
                    elif self.verification_passed:
                        display_text = "Liveness Status: Not Passed!"
                        text_color = (0, 255, 255)

            cv.putText(frame, display_text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)
            cv.imshow("AI Verification System", frame)

            if cv.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv.destroyAllWindows()


In [30]:
if __name__ == "__main__":
    system = FaceVerificationSystem(
        reference_image_url="https://i.postimg.cc/Gt1F6PLH/my-pic.jpg"
    )
    system.run_verification()

Fetching reference image from: https://i.postimg.cc/Gt1F6PLH/my-pic.jpg
